# WZK - Zadanie 1

Autor: Dariusz Max Adamski (nr indeksu 136674)

---

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import IntText, Text
import ipywidgets as widgets
from numpy.random import randint
import numpy as np
map = lambda xs, f: [f(x) for x in xs]
join = lambda xs, sep: sep.join(xs)

def mod(a, b):
    # Graham, Knuth, Patashnik - ''Concrete Mathematics''
    return a - b * (a // b)

## 1. Metoda trywialna

### Parametry

- `n` - liczba udziałów
- `k` - górna granica sekretu

### Pytania

*Jakie są podstawowe wady metody trywialnego podziału sekretu?*

Wymagane są wszystkie udziały do odtworzenia sekretu. Przez to metoda może być użyta tylko w sytuacjach gdzie do odczytania sekretu potrzebna jest zgoda wszystkich udziałowców.

Przykładowa kłopotliwa sytuacja: może być 100 udziałów i mamy zgodę wszystkich na odczytanie sekretu, ale 1 udziałowiec zgubił swój udział i w efekcie nie można odczytać sekretu.

In [2]:
@interact_manual(secret=IntText(1998), k=IntText(5000), n=(1, 10, 1))
def trivial_encrypt(secret, k, n=2):
    assert k > secret, f'k > secret not satisfied ({k} <= {secret})'
    s = secret
    keys = randint(0, k, size=n)
    s -= np.sum(keys)
    s %= k
    print('encrypted =', s)
    print('keys =', ','.join(map(keys, str)))

interactive(children=(IntText(value=1998, description='secret'), IntText(value=5000, description='k'), IntSlid…

In [3]:
@interact_manual(secret=IntText(), keys=Text(), k=IntText(5000))
def trivial_decrypt(secret, keys, k):
    s = secret
    s += sum(map(keys.split(','), int))
    s %= k
    print('decrypted =', s)

interactive(children=(IntText(value=0, description='secret'), Text(value='', description='keys'), IntText(valu…

## 2. Schemat Shamira

### Parametry
    
- `p` liczba pierwsza
- `t` wymagana liczba udziałów
- `n` całkowita liczba udziałów

### Pytania

*Jaka jest minimalna wymagana liczba udziałów, aby algorytm działał poprawnie?*

Minimalne `t` wynosi 2, ponieważ nie może być mniejsze od 1, a gdy wynosi 1, punkty `y` są równe sekretowi `s`, co sprawia, że podział sekretu jest bez sensu.

In [4]:
# proszę tu ustawić liczbę pierwszą
p = 2**13 - 1
p

8191

In [6]:
def poly_eval(A, x):
    X = np.array([x**i for i in range(len(A))])
    y = np.sum(A*X)
    return y

@interact_manual(secret=IntText(), n=(1, 10, 1), t=(1, 10, 1))
def shamir_encrypt(secret, n=4, t=3):
    assert p > secret, f'p > secret not satisfied ({p} <= {secret})'
    assert p > n, f'p > n not satisfied ({p} <= {n})'
    A = [secret] + list(randint(0, p, size=t-1))
    X = np.arange(n) + 1
    Y = [poly_eval(A, x) for x in X]
    
    # pretty print results
    for i, x in enumerate(A): print(f'a_{i} = {x}')
    for x, y in zip(X, Y): print(f's_{x} = f({x}) mod p = {y: 8d} mod {p} = {y % p}')
    print('--- ENCRYPTED ---')
    print(join([f'{x},{y % p}' for x, y in zip(X, Y)], ';'))

interactive(children=(IntText(value=0, description='secret'), IntSlider(value=4, description='n', max=10, min=…

In [7]:
from fractions import Fraction

def lagr_interp(X):
    num, den = np.ones_like(X), np.ones_like(X)
    for i, xi in enumerate(X):
        for j, xj in enumerate(X):
            if i == j: continue
            num[i] *= xj
            den[i] *= xj - xi
    return [Fraction(a, b) for a, b in zip(num, den)]

@interact_manual(keys=Text())
def shamir_decrypt(keys):
    X, Y = np.array([map(x.split(','), int) for x in keys.split(';')]).T
    L = lagr_interp(X)
    S = np.sum(Y * L) % p # modulo is distributive
    
    # pretty print results
    for i in range(len(X)): print(f'y_{i}*l_{i} = {Y[i]: 6d} * {L[i]} = {Y[i] * L[i]}')
    print('--- DECRYPTED ---')
    print(S)

interactive(children=(Text(value='', description='keys'), Button(description='Run Interact', style=ButtonStyle…